## Import packages

In [1]:
from math import pi
import matplotlib.pyplot as plt
from pennylane import numpy as np
import pennylane as qml
from pennylane.optimize import AdamOptimizer
import datetime
now=datetime.datetime.now

### Define the objective function

The target function is $f(x_1,x_2,x_3)=0.5\sin(x_1)\sin(x_2)-0.6\cos(x_2)\sin(x_3)+\cos^2(x_3)$.

In [2]:
coef=1
def my_objective(X1,X2,X3):
    Y=0.5*np.sin(coef*X1)*np.sin(coef*X2)-0.6*np.cos(coef*X2)*np.sin(coef*X3)+np.cos(coef*X3)**2
    return Y

In [3]:
my_objective(0,0,pi/2)

-0.6

## Generate data

In [4]:
np.random.seed(0)
X1,X2,X3=np.meshgrid(np.linspace(-0.95,0.95,10),np.linspace(-0.95,0.95,10),np.linspace(-0.95,0.95,10))

In [5]:
print(X1.shape)
print(X2.shape)
print(X3.shape)

(10, 10, 10)
(10, 10, 10)
(10, 10, 10)


In [6]:
X1_flatten=X1.flatten()
X2_flatten=X2.flatten()
X3_flatten=X3.flatten()

In [7]:
print(X1_flatten.shape)
print(X2_flatten.shape)
print(X3_flatten.shape)

(1000,)
(1000,)
(1000,)


In [8]:
X=np.concatenate((X1_flatten.reshape(-1,1), X2_flatten.reshape(-1,1), X2_flatten.reshape(-1,1)), axis=1)
print(X.shape)

(1000, 3)


In [9]:
Y=my_objective(X[:,0],X[:,1],X[:,2])
print(Y.shape)
print(Y[0:5])

(1000,)
[0.95306763 0.95306763 0.95306763 0.95306763 0.95306763]


## Set device

In [10]:
num_qubits=4
dev=qml.device('default.qubit', wires=num_qubits)

## Define embedding layer

In [11]:
# define my own embedding layer
def myembedding(x,wires):
    qml.RY(coef*x[0], wires=wires[1])
    qml.RY(coef*x[1], wires=wires[2])
    qml.RY(coef*x[2], wires=wires[3])

## Define the Hamiltonian matrix transformation layer

In [12]:
def Ham():
    obs=[]
    for j in range(num_qubits):
        obs.append(qml.PauliX(j))
        for k in range(j):
            obs.append(qml.PauliZ(j)@qml.PauliZ(k))
    coeffs=np.random.uniform(-1,1,len(obs))*10
    qml.Hamiltonian(coeffs, obs)

## Define ansatze layer

In [13]:
# define ansastz layer
def layer(theta):
    
    # Apply Hamiltonian matrix
    Ham()
    
    # Apply H gate
    qml.Hadamard(0)
    
    # rotations on qubit 1
    qml.RY(theta[0],wires=1)
    
    # rotations on qubit 2
    qml.RY(theta[1],wires=2)
    
    # rotations on qubit 3
    qml.RY(theta[2],wires=3)
    
    # CNOT
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[0, 2])
    qml.CNOT(wires=[0, 3])

In [14]:
@qml.qnode(dev)
def quantum_net(theta,x):
    
    # encode data
    myembedding(x,wires=range(num_qubits))
    
    # parameterized circuit layers
    for v in theta: # (for lool along with the first dimension)
        # print(v)
        # Ham()
        layer(v)
    
    qml.Hadamard(0)
    
    return qml.expval(qml.PauliZ(0)),qml.expval(qml.PauliZ(3))

In [15]:
num_layers = 4
theta = np.random.uniform(0,2*pi, size=(num_layers,num_qubits-1), requires_grad=True)
print(theta.shape)
print(theta)

(4, 3)
[[3.44829694 4.49366732 3.78727399]
 [3.42360201 2.66190161 4.0582724 ]
 [2.74944154 5.60317502 6.0548717 ]
 [2.40923412 4.97455513 3.32314479]]


In [16]:
quantum_net(theta,[0,0,0])

tensor([-0.39012416, -0.51561313], requires_grad=True)

In [17]:
print(qml.draw(quantum_net)(theta,[0,0,0]))

0: ──H──────────────────╭C─╭C─╭C──H────────╭C─╭C─╭C──H────────╭C─╭C─╭C──H────────╭C─╭C─╭C──H─┤  <Z>
1: ──RY(0.00)──RY(3.45)─╰X─│──│───RY(3.42)─╰X─│──│───RY(2.75)─╰X─│──│───RY(2.41)─╰X─│──│─────┤     
2: ──RY(0.00)──RY(4.49)────╰X─│───RY(2.66)────╰X─│───RY(5.60)────╰X─│───RY(4.97)────╰X─│─────┤     
3: ──RY(0.00)──RY(3.79)───────╰X──RY(4.06)───────╰X──RY(6.05)───────╰X──RY(3.32)───────╰X────┤  <Z>


## Add classical layer

In [18]:
# add the classical layer
def classical_quantum_net(theta,w,x):
    r1=quantum_net(theta,x)[0]
    r2=quantum_net(theta,x)[1]
    return w[0]+w[1]*r1+w[2]*r1**2+w[3]*r2+w[4]*r2**2

In [19]:
def square_loss(labels,predictions):
    loss=0
    for l,p in zip(labels,predictions):
        loss=loss+(l-p)**2
    loss=loss/len(labels)
    return loss

In [20]:
def cost(theta,w,features,labels):
    preds=[classical_quantum_net(theta,w,x) for x in features]
    return square_loss(labels,preds)

## Model training

In [21]:
w=np.zeros(5,requires_grad=True)

Using the Adam optimizer, we update the weights for 200 steps (this takes some time). More steps will lead to a better fit.

In [22]:
opt = AdamOptimizer(0.05, beta1=0.9, beta2=0.999)

In [23]:
start=now()
print(start)

2022-08-07 16:58:57.814722


In [24]:
epochs=200
for e in range(1,epochs+1):
    
    (theta,w,_,_),_cost=opt.step_and_cost(cost,theta,w,X,Y)

    if e==1 or e%10==0:
        print(f'Epoch: {e} | Cost: {_cost} | w: {w}')

Epoch: 1 | Cost: 0.6292084922407113 | w: [ 0.04999999 -0.04999996  0.04999989 -0.04999998  0.04999996]
Epoch: 10 | Cost: 0.10201701733734839 | w: [ 0.38142472 -0.35895746  0.33523682 -0.39631322  0.40135958]
Epoch: 20 | Cost: 0.049997612201151724 | w: [ 0.44961766 -0.3554002   0.23413232 -0.44706589  0.40399351]
Epoch: 30 | Cost: 0.03785023812550846 | w: [ 0.44053348 -0.48311358  0.13186744 -0.4850311   0.42910152]
Epoch: 40 | Cost: 0.022740111739543132 | w: [ 0.39877363 -0.65860213  0.0345266  -0.51903668  0.46165487]
Epoch: 50 | Cost: 0.011006203267384699 | w: [ 0.35455036 -0.82917983 -0.05354515 -0.54815334  0.49595177]
Epoch: 60 | Cost: 0.004760732997609078 | w: [ 0.32367232 -0.96636639 -0.12564107 -0.56957551  0.52704464]
Epoch: 70 | Cost: 0.0021741830042198884 | w: [ 0.30436324 -1.04955555 -0.17537453 -0.58592877  0.55650509]
Epoch: 80 | Cost: 0.0020828132007480377 | w: [ 0.29490461 -1.08146895 -0.19430555 -0.59398237  0.57242125]
Epoch: 90 | Cost: 0.0020130117430375793 | w: [ 0.

In [25]:
end=now()
print(end)

2022-08-07 19:50:54.098753


In [26]:
print(end-start)

2:51:56.284031


## Training error

In [27]:
pred_train=[classical_quantum_net(theta,w,x) for x in X]

In [28]:
train_diff=np.abs(Y-pred_train)

In [29]:
np.max(train_diff)

tensor(0.12937399, requires_grad=True)

In [30]:
np.min(train_diff)

tensor(0.00017072, requires_grad=True)

In [31]:
np.mean(train_diff)

tensor(0.02710076, requires_grad=True)

## Test error

In [32]:
test1,test2,test3=np.meshgrid(np.linspace(-0.95,0.95,30),np.linspace(-0.95,0.95,30),np.linspace(-0.95,0.95,30))
test1_flatten=test1.flatten()
test2_flatten=test2.flatten()
test3_flatten=test3.flatten()
Y_test=my_objective(test1_flatten,test2_flatten,test3_flatten)
pred_test=[classical_quantum_net(theta,w,x) for x in zip(test1_flatten,test2_flatten,test3_flatten)]

In [33]:
test_diff=np.abs(Y_test-pred_test)

In [34]:
np.max(test_diff)

tensor(0.72830791, requires_grad=True)

In [35]:
np.min(test_diff)

tensor(6.45413797e-06, requires_grad=True)

In [36]:
np.mean(test_diff)

tensor(0.14827442, requires_grad=True)

In [37]:
print(theta)

[[4.13352456 4.97473187 3.44633472]
 [3.61175457 3.09902118 3.45681543]
 [2.42842127 6.559291   6.75058266]
 [2.63810189 4.79082253 3.25993819]]


## Randomly generate test dataset

In [38]:
number=27000
Test=np.random.uniform(-0.95,0.95,size=(number,3))

In [39]:
Test.shape

(27000, 3)

In [40]:
Y_Test=my_objective(Test[:,0],Test[:,1],Test[:,2])
print(Y_test.shape)

(27000,)


In [41]:
pred_Test=[classical_quantum_net(theta,w,x) for x in Test]

In [42]:
Test_diff=np.abs(Y_Test-pred_Test)

In [43]:
np.max(Test_diff)

tensor(0.71634444, requires_grad=True)

In [44]:
np.min(Test_diff)

tensor(3.92253809e-06, requires_grad=True)

In [45]:
np.mean(Test_diff)

tensor(0.14276355, requires_grad=True)